# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*



# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*


**This file inlcudes the method to fine-tune the pre-trained Lanaguage Model**

**This method needs to train.**

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import os

# Set paths
data_dir = "/content/drive/MyDrive/90042 a3/COMP90042_2025/data/"
train_claims_file = os.path.join(data_dir, "train-claims.json")
dev_claims_file = os.path.join(data_dir, "dev-claims.json")
test_claims_file = os.path.join(data_dir, "test-claims-unlabelled.json")
evidence_file = os.path.join(data_dir, "evidence.json")
# hard_negatives_file = os.path.join(data_dir, "hard_negatives.json")

# Load train claims
with open(train_claims_file, 'r') as f:
    train_claims = json.load(f)
train_ids = list(train_claims.keys())
train_texts = [train_claims[claim_id]['claim_text'] for claim_id in train_ids]
claim_id_to_train_inidce = {claim_id: i for i, claim_id in enumerate(train_ids)}

print(f"Loaded {len(train_claims)} train claims.")

# Load dev claims
with open(dev_claims_file, 'r') as f:
    dev_claims = json.load(f)
dev_ids = list(dev_claims.keys())
dev_texts = [dev_claims[claim_id]['claim_text'] for claim_id in dev_ids]

print(f"Loaded {len(dev_claims)} dev claims.")

# Load test claims
with open(test_claims_file, 'r') as f:
    test_claims = json.load(f)
test_texts = [test_claims[claim_id]['claim_text'] for claim_id in test_claims.keys()]

print(f"Loaded {len(test_claims)} test claims.")

# Load evidence texts
with open(evidence_file, 'r') as f:
    evidence = json.load(f)

# Load pre-computed hard negatives
with open(os.path.join(data_dir, "hard_negatives.json"), 'r') as f:
    claim_to_hard_negatives = json.load(f)

evidence_ids = list(evidence.keys())
evidence_texts = [evidence[claim_id] for claim_id in evidence_ids]
evidence_id_to_train_index = {claim_id: i for i, claim_id in enumerate(evidence_ids)}
print(f"Loaded {len(evidence)} evidence documents.")

Loaded 1228 train claims.
Loaded 154 dev claims.
Loaded 153 test claims.
Loaded 1208827 evidence documents.


In [3]:
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"

import torch

from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)
# model = AutoModel.from_pretrained(model_name).cuda()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [5]:
def contrastive_loss(claim_embedding, pos_evidence_embeddings, neg_evidence_embeddings, temperature=0.1):
    """Compute contrastive loss for a claim"""
    # Compute positive and negative similarities
    pos_sim = torch.exp(torch.matmul(claim_embedding, pos_evidence_embeddings.T) / temperature).sum()
    neg_sim = torch.exp(torch.matmul(claim_embedding, neg_evidence_embeddings.T) / temperature).sum()

    # Compute contrastive loss
    loss = -torch.log(pos_sim / (pos_sim + neg_sim))
    return loss

In [16]:
def random_batch_update_hard_negatives(model, train_claims, evidence_texts, evidence_ids, tokenizer, num_claims=100, top_k=20):
    """Randomly update hard negatives for a subset of claims"""
    print("Updating hard negatives for random subset of claims...")
    model.eval()

    # Randomly select claims to update
    claim_ids = list(train_claims.keys())
    selected_claim_ids = np.random.choice(claim_ids, size=num_claims, replace=False)

    # Update hard negatives only for selected claims
    new_hard_negatives = {}
    batch_size = 32

    for i in tqdm(range(0, len(selected_claim_ids), batch_size)):
        batch_ids = selected_claim_ids[i:i + batch_size]
        batch_claims = [train_claims[cid] for cid in batch_ids]

        # Encode claims
        claim_texts = [claim['claim_text'] for claim in batch_claims]
        with torch.no_grad():
            claim_inputs = tokenizer(claim_texts, padding=True,
                                   truncation=True, return_tensors='pt').to(model.device)
            claim_embeddings = model(**claim_inputs).last_hidden_state[:, 0, :]
            claim_embeddings = F.normalize(claim_embeddings, p=2, dim=1)

        # Randomly sample evidence for comparison
        num_samples = 1000
        sampled_indices = np.random.choice(len(evidence_texts), size=num_samples, replace=False)
        sampled_texts = [evidence_texts[idx] for idx in sampled_indices]

        # Encode sampled evidence
        with torch.no_grad():
            evidence_inputs = tokenizer(sampled_texts, padding=True,
                                     truncation=True, return_tensors='pt').to(model.device)
            evidence_embeddings = model(**evidence_inputs).last_hidden_state[:, 0, :]
            evidence_embeddings = F.normalize(evidence_embeddings, p=2, dim=1)

            # Calculate similarities
            similarities = torch.mm(claim_embeddings, evidence_embeddings.t())

            # Get top k for each claim
            top_k_sims, top_k_indices = torch.topk(similarities, k=top_k)

            # Store results
            for claim_idx, claim_id in enumerate(batch_ids):
                hard_negatives = []
                for sim_idx, sim_val in zip(top_k_indices[claim_idx].cpu().numpy(),
                                          top_k_sims[claim_idx].cpu().numpy()):
                    evidence_id = evidence_ids[sampled_indices[sim_idx]]
                    if evidence_id not in batch_claims[claim_idx]['evidences']:
                        hard_negatives.append({
                            'evidence_id': evidence_id,
                            'similarity': float(sim_val)
                        })
                new_hard_negatives[claim_id] = hard_negatives[:top_k]

        torch.cuda.empty_cache()

    model.train()
    return new_hard_negatives


In [9]:
"""
test code
It will calcualte the score in dev dataset.
"""
from tqdm import tqdm
import torch.nn.functional as F

def generate_all_embeddings(model, texts, batch_size=32):
    """Generate embeddings for all texts"""
    model.eval()
    all_embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i + batch_size]
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt').to(model.device)
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :].cpu()  # CLS token
            all_embeddings.append(embeddings)
            torch.cuda.empty_cache()
    return torch.cat(all_embeddings)


import numpy as np
def get_test_f_scores(test_texts, test_claims, test_ids, evidence_texts, evidence_indice_to_claim_id, model, batch_size=32):
    """Get F-scores for test claims"""
    model.eval()

    # Generate test claim embeddings
    test_claim_embeddings = generate_all_embeddings(model, test_texts, batch_size=batch_size)

    # Generate evidence embeddings
    evidence_embeddings = generate_all_embeddings(model, evidence_texts, batch_size=batch_size*10)

    # Compute cosine similarities
    norm_test_claim_embeddings = F.normalize(test_claim_embeddings, p=2, dim=1)
    norm_evidence_embeddings = F.normalize(evidence_embeddings, p=2, dim=1)
    similarities = torch.matmul(norm_test_claim_embeddings, norm_evidence_embeddings.T)

    # Get top-k evidence indices for each claim
    top_k = 5
    top_k_indices = torch.topk(similarities, top_k, dim=1).indices
    top_k_indices = top_k_indices.numpy()

    # Compute F-scores
    f_scores = []
    for i, claim_id in enumerate(test_ids):
        # Get the evidence indices for the claim
        true_evidence_indices = test_claims[claim_id]["evidences"]

        # Get the predicted evidence indices
        predicted_evidence_indices = top_k_indices[i]
        predicted_evidence_indices = [evidence_indice_to_claim_id[evidence_index] for evidence_index in predicted_evidence_indices]

        # Compute precision and recall
        true_positives = len(set(true_evidence_indices) & set(predicted_evidence_indices))
        precision = true_positives / len(predicted_evidence_indices) if len(predicted_evidence_indices) > 0 else 0.0
        recall = true_positives / len(true_evidence_indices) if len(true_evidence_indices) > 0 else 0.0

        # Compute F-score
        f_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        f_scores.append(f_score)
    model.train()
    return np.mean(f_scores)

In [26]:
# Training configuration
max_epochs = 10
batch_size = 32
learning_rate = 1e-5
test_interval = 100  # Test every 100 steps
log_interval = 20    # Log every 20 steps
update_frequency = 2 # Update hard negatives every 2 epochs
best_f1 = 0

# Initialize model
model = AutoModel.from_pretrained(model_name)
model.cuda()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)

# Set random seed for reproducibility
random.seed(330)
torch.manual_seed(330)
torch.cuda.manual_seed_all(330)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load pre-computed TF-IDF hard negatives
with open(os.path.join(data_dir, "hard_negatives.json"), 'r') as f:
    tfidf_hard_negatives = json.load(f)

# Initialize hard negatives dictionary with TF-IDF results
hard_negatives_dict = tfidf_hard_negatives.copy()

step = 0
max_score = 0.0

for epoch in range(max_epochs):
    print(f"Epoch {epoch + 1}/{max_epochs}")
    random.shuffle(train_ids)

    if step > 120:
        break

    # Alternate between TF-IDF and model-based hard negatives
    use_tfidf = (epoch % (update_frequency * 2) == 0)  # True for epochs 0, 4, 8...
    use_model = (epoch % update_frequency == 0 and not use_tfidf)  # True for epochs 2, 6...

    if use_tfidf:
        print(f"Epoch {epoch + 1}: Using TF-IDF hard negatives")
        hard_negatives_dict = tfidf_hard_negatives.copy()

    elif use_model:
        print(f"Epoch {epoch + 1}: Updating hard negatives using current model")
        model.eval()
        with torch.no_grad():
            # Filter TF-IDF hard negatives using current model
            for claim_id in tqdm(train_claims.keys(), desc="Filtering hard negatives"):
                # Get claim embedding
                claim_text = train_claims[claim_id]['claim_text']
                inputs = tokenizer(claim_text, padding=True, truncation=True, return_tensors='pt').to(model.device)
                claim_embedding = model(**inputs).last_hidden_state[:, 0, :]
                norm_claim_embedding = F.normalize(claim_embedding, p=2, dim=1)

                # Get TF-IDF candidates
                tfidf_candidates = tfidf_hard_negatives[claim_id]
                candidate_evidence_ids = [neg['evidence_id'] for neg in tfidf_candidates]
                candidate_texts = [evidence[eid] for eid in candidate_evidence_ids]

                if not candidate_texts:  # Skip if no candidates
                    continue

                # Calculate similarities with current model
                inputs = tokenizer(
                    candidate_texts,
                    padding=True,
                    truncation=True,
                    max_length=256,
                    return_tensors='pt'
                ).to(model.device)

                evidence_embeddings = model(**inputs).last_hidden_state[:, 0, :]
                norm_evidence_embeddings = F.normalize(evidence_embeddings, p=2, dim=1)

                similarities = torch.matmul(norm_claim_embedding, norm_evidence_embeddings.T)
                similarities = similarities[0].cpu().numpy()

                # Get top k most similar negatives according to current model
                top_k = 5
                top_indices = np.argsort(similarities)[-top_k:]

                # Update hard negatives for this claim
                new_hard_negatives = []
                for idx in top_indices:
                    evidence_id = candidate_evidence_ids[idx]
                    if evidence_id not in train_claims[claim_id]['evidences']:
                        new_hard_negatives.append({
                            'evidence_id': evidence_id,
                            'similarity': float(similarities[idx])
                        })

                hard_negatives_dict[claim_id] = new_hard_negatives

        model.train()

    # Training loop
    for i in range(0, len(train_ids), batch_size):
        if step > 120:
            break

        step += 1
        print(f"Step {step}")
        batch_ids = train_ids[i:i + batch_size]
        batch_claims = [train_claims[claim_id] for claim_id in batch_ids]
        batch_indices = [claim_id_to_train_inidce[claim_id] for claim_id in batch_ids]

        # Get evidence indices including current hard negatives
        evidence_indices = []
        pos_evidence_positive_indices = []
        for claim_id, claim in zip(batch_ids, batch_claims):
            # Add positive samples
            positive_indices = []
            for evidence_id in claim["evidences"][:2]:  # Use at most 2 positive samples
                evidence_idx = evidence_id_to_train_index[evidence_id]
                if evidence_idx not in evidence_indices:
                    evidence_indices.append(evidence_idx)
                positive_indices.append(len(evidence_indices) - 1)

            # Add current hard negatives
            hard_negs = hard_negatives_dict[claim_id][:3]  # Use top 3 hard negatives
            for neg in hard_negs:
                neg_idx = evidence_id_to_train_index[neg['evidence_id']]
                if neg_idx not in evidence_indices:
                    evidence_indices.append(neg_idx)

            pos_evidence_positive_indices.append(positive_indices)

        # Get claim embeddings
        claim_texts = [train_texts[i] for i in batch_indices]
        model_inputs = tokenizer(claim_texts, padding=True, truncation=True, return_tensors='pt').to(model.device)
        claim_embeddings = model(**model_inputs).last_hidden_state[:, 0, :]
        norm_claim_embeddings = F.normalize(claim_embeddings, p=2, dim=1)

        # Get evidence embeddings
        cur_evidence_indices = [evidence_texts[evidence_indice] for evidence_indice in evidence_indices]
        evidence_model_inputs = tokenizer(
            cur_evidence_indices,
            padding=True,
            truncation=True,
            max_length=256,
            return_tensors='pt'
        ).to(model.device)

        evidence_embeddings = model(**evidence_model_inputs).last_hidden_state[:, 0, :]
        norm_evidence_embeddings = F.normalize(evidence_embeddings, p=2, dim=1)

        # Calculate loss
        loss = []
        for i, claim_embedding in enumerate(norm_claim_embeddings):
            pos_evidence_embeddings = norm_evidence_embeddings[torch.tensor(pos_evidence_positive_indices[i])]
            neg_evidence_embeddings = norm_evidence_embeddings[torch.tensor([j for j in range(len(evidence_indices))
                                                               if j not in pos_evidence_positive_indices[i]])]
            loss.append(contrastive_loss(claim_embedding, pos_evidence_embeddings, neg_evidence_embeddings))
        loss = torch.mean(torch.stack(loss))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Clear cache
        del evidence_embeddings, norm_evidence_embeddings
        torch.cuda.empty_cache()

        if step % log_interval == 0:
            print(f"Loss: {loss.item()}")

        if step % test_interval == 0:
            # Evaluate on dev set
            f_score = get_test_f_scores(dev_texts, dev_claims, dev_ids, evidence_texts, evidence_ids, model, batch_size=batch_size)
            print(f"F-score on dev set: {f_score}")
            # Save model
            torch.save(model.state_dict(), f"model_epoch_{epoch + 1}_step_{step}.pth")
            if f_score > max_score:
                max_score = f_score
                print(f"New best F-score: {max_score}")
                torch.save(model.state_dict(), "best_model.pth")

Epoch 1/10
Epoch 1: Using TF-IDF hard negatives
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Step 8
Step 9
Step 10
Step 11
Step 12
Step 13
Step 14
Step 15
Step 16
Step 17
Step 18
Step 19
Step 20
Loss: 2.3991665840148926
Step 21
Step 22
Step 23
Step 24
Step 25
Step 26
Step 27
Step 28
Step 29
Step 30
Step 31
Step 32
Step 33
Step 34
Step 35
Step 36
Step 37
Step 38
Step 39
Epoch 2/10
Step 40
Loss: 2.211751699447632
Step 41
Step 42
Step 43
Step 44
Step 45
Step 46
Step 47
Step 48
Step 49
Step 50
Step 51
Step 52
Step 53
Step 54
Step 55
Step 56
Step 57
Step 58
Step 59
Step 60
Loss: 1.6401870250701904
Step 61
Step 62
Step 63
Step 64
Step 65
Step 66
Step 67
Step 68
Step 69
Step 70
Step 71
Step 72
Step 73
Step 74
Step 75
Step 76
Step 77
Step 78
Epoch 3/10
Epoch 3: Updating hard negatives using current model


Filtering hard negatives: 100%|██████████| 1228/1228 [00:15<00:00, 78.84it/s]


Step 79
Step 80
Loss: 2.328768253326416
Step 81
Step 82
Step 83
Step 84
Step 85
Step 86
Step 87
Step 88
Step 89
Step 90
Step 91
Step 92
Step 93
Step 94
Step 95
Step 96
Step 97
Step 98
Step 99
Step 100
Loss: 2.0903983116149902


100%|██████████| 3778/3778 [05:21<00:00, 11.76it/s]


F-score on dev set: 0.12941146155431874
New best F-score: 0.12941146155431874
Step 101
Step 102
Step 103
Step 104
Step 105
Step 106
Step 107
Step 108
Step 109
Step 110
Step 111
Step 112
Step 113
Step 114
Step 115
Step 116
Step 117
Epoch 4/10
Step 118
Step 119
Step 120
Loss: 2.0230536460876465
Step 121
Step 122
Step 123
Step 124
Step 125
Step 126
Step 127
Step 128
Step 129
Step 130
Step 131
Step 132
Step 133
Step 134
Step 135
Step 136
Step 137
Step 138
Step 139
Step 140
Loss: 2.290450096130371
Step 141
Step 142
Step 143
Step 144
Step 145
Step 146
Step 147
Step 148
Step 149
Step 150
Step 151
Step 152
Step 153
Step 154
Step 155
Step 156
Epoch 5/10
Epoch 5: Using TF-IDF hard negatives
Step 157
Step 158
Step 159
Step 160
Loss: 1.7610337734222412
Step 161
Step 162
Step 163
Step 164
Step 165
Step 166
Step 167
Step 168
Step 169
Step 170
Step 171
Step 172
Step 173
Step 174
Step 175
Step 176
Step 177
Step 178
Step 179
Step 180
Loss: 1.9023964405059814
Step 181
Step 182
Step 183
Step 184
Step 185

...

F-score on dev set: 0.13046794475365905
New best F-score: 0.13046794475365905
Step 201
Step 202
Step 203
Step 204
Step 205
Step 206
Step 207
Step 208
Step 209
Step 210
Step 211
Step 212
Step 213
Step 214
Step 215
Step 216
Step 217
Step 218
Step 219
Step 220
Loss: 1.6834733486175537
Step 221
Step 222
Step 223
Step 224
Step 225
Step 226
Step 227
Step 228
Step 229
Step 230
Step 231
Step 232
Step 233
Step 234
Epoch 7/10
Epoch 7: Updating hard negatives using current model


Filtering hard negatives: 100%|██████████| 1228/1228 [00:15<00:00, 79.68it/s]


Step 235
Step 236
Step 237
Step 238
Step 239
Step 240
Loss: 1.427680253982544
Step 241
Step 242
Step 243
Step 244
Step 245
Step 246
Step 247
Step 248
Step 249
Step 250
Step 251
Step 252
Step 253
Step 254
Step 255
Step 256
Step 257
Step 258
Step 259
Step 260
Loss: 1.9668530225753784
Step 261
Step 262
Step 263
Step 264
Step 265
Step 266
Step 267
Step 268
Step 269
Step 270
Step 271
Step 272
Step 273
Epoch 8/10
Step 274
Step 275
Step 276
Step 277
Step 278
Step 279
Step 280
Loss: 1.6627836227416992
Step 281
Step 282
Step 283
Step 284
Step 285
Step 286
Step 287
Step 288
Step 289
Step 290
Step 291
Step 292
Step 293
Step 294
Step 295
Step 296
Step 297
Step 298
Step 299
Step 300
Loss: 1.6331028938293457


100%|██████████| 3778/3778 [05:21<00:00, 11.74it/s]


F-score on dev set: 0.1440527726242012
New best F-score: 0.1440527726242012
Step 301
Step 302
Step 303
Step 304
Step 305
Step 306
Step 307
Step 308
Step 309
Step 310
Step 311
Step 312
Epoch 9/10
Epoch 9: Using TF-IDF hard negatives
Step 313
Step 314
Step 315
Step 316
Step 317
Step 318
Step 319
Step 320
Loss: 1.6436922550201416
Step 321
Step 322
Step 323
Step 324
Step 325
Step 326
Step 327
Step 328
Step 329
Step 330
Step 331
Step 332
Step 333
Step 334
Step 335
Step 336
Step 337
Step 338
Step 339
Step 340
Loss: 1.3321263790130615
Step 341
Step 342
Step 343
Step 344
Step 345
Step 346
Step 347
Step 348
Step 349
Step 350
Step 351
Epoch 10/10
Step 352
Step 353
Step 354
Step 355
Step 356
Step 357
Step 358
Step 359
Step 360
Loss: 1.455015778541565
Step 361
Step 362
Step 363
Step 364
Step 365
Step 366
Step 367
Step 368
Step 369
Step 370
Step 371
Step 372
Step 373
Step 374
Step 375
Step 376
Step 377
Step 378
Step 379
Step 380
Loss: 1.276928186416626
Step 381
Step 382
Step 383
Step 384
Step 385
S

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [27]:
# Load the best model "best_model.pth"
model.load_state_dict(torch.load("best_model.pth", weights_only=True))
model.eval()

test_claims_embeddings = generate_all_embeddings(model, test_texts, batch_size=batch_size)
test_evidence_embeddings = generate_all_embeddings(model, evidence_texts, batch_size=batch_size*10)

norm_test_claim_embeddings = F.normalize(test_claims_embeddings, p=2, dim=1)
norm_evidence_embeddings = F.normalize(test_evidence_embeddings, p=2, dim=1)

# Compute cosine similarities
similarities = torch.matmul(norm_test_claim_embeddings, norm_evidence_embeddings.T)

# Get top-k evidence indices for each claim
top_k = 5
top_k_indices = torch.topk(similarities, top_k, dim=1).indices
top_k_indices = top_k_indices.numpy()

# Save the results
test_claims_ids = list(test_claims.keys())
results = test_claims
for i, claim_id in enumerate(test_claims_ids):
    # Get the evidence indices for the claim
    predicted_evidence_indices = top_k_indices[i]
    predicted_evidence_indices = [evidence_ids[evidence_index] for evidence_index in predicted_evidence_indices]

    # Store the results
    results[claim_id]['evidences'] = predicted_evidence_indices


# Save the results to a JSON file
output_file = os.path.join(data_dir, "test_claims_retrieved_bert.json")
with open(output_file, 'w') as f:
    json.dump(results, f, indent=4)

100%|██████████| 3778/3778 [05:21<00:00, 11.77it/s]


## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*